<a href="https://colab.research.google.com/github/dariakrtn/AI_2022/blob/main/lab_4_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. *Импорт библиотек и вспомогательные функции/классы*

In [2]:
# !pip install optuna

In [3]:
import optuna
import sklearn.ensemble
import sklearn.svm
import logging

In [4]:
import logging
import sys

import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import optuna

In [5]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

# 1. *Применение оптуны на функцию Матьяса*

In [6]:
def f_mat (x, y):
  return 0.26*(x*x +y*y)-0.48*x*y

def objective(trial):
  x = trial.suggest_float('x', -10, 10)
  y = trial.suggest_float('y', -10, 10)
  return f_mat (x, y)

In [7]:
optuna.logging.disable_default_handler() 
optuna.logging.disable_propagation()
study =optuna.create_study()
study.optimize(objective, n_trials=100)


In [8]:
best_params = study.best_params
found_x = best_params['x']
found_y = best_params['y']
print(f"Found x: {found_x},  Found y: {found_y}, f_mat(x,y): {f_mat(found_x, found_y)}")

Found x: 0.2812734833141193,  Found y: 0.19611118336715183, f_mat(x,y): 0.004092115532888199


# 2. *Использование datasets Iris и оптимизация параметров с помощью optuna*

In [9]:
def objective(trial):
    iris = sklearn.datasets.load_iris()  # грузим датасет
    classes = list(set(iris.target))
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split( # разюитие на тест и тренировку в данном случае 75 на 25
        iris.data, iris.target, test_size=0.25, random_state=0
    )


    max_iter = trial.suggest_int("max_iter", 50, 400)
    verbose = trial.suggest_int("verbose", 0, 100)
    clf = sklearn.linear_model.SGDClassifier(max_iter=max_iter, verbose = verbose)

    for step in range(100):
        clf.fit(train_x, train_y)

        # Report intermediate objective value.
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()

    return 1.0 - clf.score(valid_x, valid_y)


In [10]:
optuna.logging.disable_default_handler() 
optuna.logging.disable_propagation()
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

Выходные данные были обрезаны до нескольких последних строк (5000).
-- Epoch 4
Norm: 67.54, NNZs: 4, Bias: 9.960396, T: 448, Avg. loss: 0.000000
Total training time: 0.02 seconds.
-- Epoch 5
Norm: 62.68, NNZs: 4, Bias: 9.960396, T: 560, Avg. loss: 0.000000
Total training time: 0.02 seconds.
-- Epoch 6
Norm: 58.48, NNZs: 4, Bias: 9.960396, T: 672, Avg. loss: 0.000000
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 54.81, NNZs: 4, Bias: 9.960396, T: 784, Avg. loss: 0.000000
Total training time: 0.02 seconds.
Convergence after 7 epochs took 0.02 seconds
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
-- Epoch 1
Norm: 115.16, NNZs: 4, Bias: 0.439006, T: 112, Avg. loss: 135.968696
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 118.78, NNZs: 4, Bias: 17.729531, T: 224, Avg. loss: 121.275901
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 125.75, NNZs: 4, Bias: 40.774707, T: 336, Avg. loss: 108.694694
Total training time: 0.00 seconds.
-- Epoch 4
N

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0

Norm: 131.89, NNZs: 4, Bias: 42.869864, T: 672, Avg. loss: 96.345681
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 143.10, NNZs: 4, Bias: 54.461805, T: 784, Avg. loss: 69.069111
Total training time: 0.01 seconds.
-- Epoch 8
Norm: 161.14, NNZs: 4, Bias: 65.307186, T: 896, Avg. loss: 81.686615
Total training time: 0.01 seconds.
-- Epoch 9
Norm: 154.91, NNZs: 4, Bias: 70.533850, T: 1008, Avg. loss: 54.024479
Total training time: 0.01 seconds.
-- Epoch 10
Norm: 165.25, NNZs: 4, Bias: 75.213494, T: 1120, Avg. loss: 71.653405
Total training time: 0.01 seconds.
-- Epoch 11
Norm: 169.59, NNZs: 4, Bias: 74.966855, T: 1232, Avg. loss: 50.265180
Total training time: 0.01 seconds.
-- Epoch 12
Norm: 174.70, NNZs: 4, Bias: 74.972593, T: 1344, Avg. loss: 50.940571
Total training time: 0.01 seconds.
-- Epoch 13
Norm: 179.91, NNZs: 4, Bias: 75.024686, T: 1456, Avg. loss: 48.358129
Total training time: 0.01 seconds.
-- Epoch 14
Norm: 170.85, NNZs: 4, Bias: 87.081832, T: 1568, Avg. loss: 44.317926


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.

Norm: 266.59, NNZs: 4, Bias: -63.753518, T: 336, Avg. loss: 31.152597
Total training time: 0.06 seconds.
-- Epoch 4
Norm: 278.47, NNZs: 4, Bias: -63.968401, T: 448, Avg. loss: 22.741927
Total training time: 0.12 seconds.
-- Epoch 5
Norm: 307.10, NNZs: 4, Bias: -77.597225, T: 560, Avg. loss: 32.114728
Total training time: 0.12 seconds.
-- Epoch 6
Norm: 313.77, NNZs: 4, Bias: -96.058663, T: 672, Avg. loss: 17.495646
Total training time: 0.13 seconds.
-- Epoch 7
Norm: 308.19, NNZs: 4, Bias: -96.029476, T: 784, Avg. loss: 12.965282
Total training time: 0.13 seconds.
-- Epoch 8
Norm: 295.06, NNZs: 4, Bias: -101.330013, T: 896, Avg. loss: 5.000081
Total training time: 0.13 seconds.
-- Epoch 9
Norm: 288.15, NNZs: 4, Bias: -101.254069, T: 1008, Avg. loss: 9.768004
Total training time: 0.13 seconds.
-- Epoch 10
Norm: 287.11, NNZs: 4, Bias: -101.259094, T: 1120, Avg. loss: 9.251433
Total training time: 0.13 seconds.
-- Epoch 11
Norm: 299.10, NNZs: 4, Bias: -110.522386, T: 1232, Avg. loss: 15.469

...


Norm: 115.76, NNZs: 4, Bias: 9.863063, T: 112, Avg. loss: 14.055628
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 105.16, NNZs: 4, Bias: 9.863063, T: 224, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 96.34, NNZs: 4, Bias: 9.863063, T: 336, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 88.88, NNZs: 4, Bias: 9.863063, T: 448, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 82.49, NNZs: 4, Bias: 9.863063, T: 560, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 76.96, NNZs: 4, Bias: 9.863063, T: 672, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 72.13, NNZs: 4, Bias: 9.863063, T: 784, Avg. loss: 0.000000
Total training time: 0.00 seconds.
Convergence after 7 epochs took 0.00 seconds
-- Epoch 1
Norm: 93.53, NNZs: 4, Bias: -0.669394, T: 112, Avg. loss: 135.536905
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 134.30, NNZs: 4, Bias: 7.784195, T: 224, Avg. 

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


-- Epoch 1
Norm: 103.15, NNZs: 4, Bias: 9.764019, T: 112, Avg. loss: 13.051549
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 93.71, NNZs: 4, Bias: 9.764019, T: 224, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 85.85, NNZs: 4, Bias: 9.764019, T: 336, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 79.20, NNZs: 4, Bias: 9.764019, T: 448, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 73.51, NNZs: 4, Bias: 9.764019, T: 560, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 68.58, NNZs: 4, Bias: 9.764019, T: 672, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 64.28, NNZs: 4, Bias: 9.764019, T: 784, Avg. loss: 0.000000
Total training time: 0.00 seconds.
Convergence after 7 epochs took 0.00 seconds
-- Epoch 1
Norm: 141.90, NNZs: 4, Bias: -9.244449, T: 112, Avg. loss: 135.198846
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 104.08, NNZs: 4, Bias: 16.552033, T:

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


Выходные данные были обрезаны до нескольких последних строк (5000).
Total training time: 0.00 seconds.
-- Epoch 11
Norm: 165.42, NNZs: 4, Bias: 82.182320, T: 1232, Avg. loss: 55.707865
Total training time: 0.00 seconds.
-- Epoch 12
Norm: 166.80, NNZs: 4, Bias: 86.565016, T: 1344, Avg. loss: 48.449035
Total training time: 0.00 seconds.
-- Epoch 13
Norm: 171.57, NNZs: 4, Bias: 86.593408, T: 1456, Avg. loss: 47.125307
Total training time: 0.00 seconds.
-- Epoch 14
Norm: 176.28, NNZs: 4, Bias: 90.691587, T: 1568, Avg. loss: 42.853642
Total training time: 0.00 seconds.
-- Epoch 15
Norm: 165.53, NNZs: 4, Bias: 98.284572, T: 1680, Avg. loss: 46.135701
Total training time: 0.00 seconds.
-- Epoch 16
Norm: 171.78, NNZs: 4, Bias: 94.584105, T: 1792, Avg. loss: 39.690480
Total training time: 0.00 seconds.
-- Epoch 17
Norm: 164.14, NNZs: 4, Bias: 105.071796, T: 1904, Avg. loss: 42.613100
Total training time: 0.00 seconds.
-- Epoch 18
Norm: 163.36, NNZs: 4, Bias: 108.431640, T: 2016, Avg. loss: 40.2

In [11]:
plot_optimization_history(study)


In [12]:

plot_contour(study)


In [13]:
plot_parallel_coordinate(study)


In [14]:
plot_slice(study)


In [15]:
plot_param_importances(study)


In [16]:
plot_edf(study)